# Урок 15. Консультация по курсовому проекту. Создание чат-бота в Telegram
##Задание

1. Продумать тему курсового проекта. 
2. Подготовить данные для обучения. 
3. Продумать интенты на которые вы будете обучаться.

**Для сдачи задания необходимо написать:**

- На какую тему ваш бот будет общаться (на каких датасетах планируете обучать модели)
- Написать на какие интенты вы будете обучать бота

**Примеры интентов**

- болталка
- погода в городе
- время в городе
- праздники
- фильмы
- покупка товара
- можно добавить что-то своё

**Из обязательных (всего обязательно 3-ри интента):**

ваш бот дожен работать в режиме
1. болталка
2. на ваш выбор
3. на ваш выбор****

Я планирую сделать телеграмм-бота, который будет работать в режиме обычной болталки, а также общение на медицинскую тематику.

**Интенты:**
1. Болталка
2. Q&A система на основе вопросов пациентов и ответов врачей (симтомы-диагноз)
3. Система подбора релевантных отзывов на лекарства.

**Датасеты:**
1. Для классификации интента "болталка" будет использованы вопросы и ответы от мейл, файл Otvety.txt
2. Для Q&A системы беру датасет https://huggingface.co/datasets/blinoff/medical_qa_ru_data. В датасете имеется 2 столбца: desc - это первый вопрос пациента, и ans - это спарсенный диалог с оговоренным разделителем сообщений. Причем в ответах есть как ответы разных врачей, так и встречные вопросы пациента. И сообщения от врачей и от пациента уже никак не помечены, это можно понять только по смыслу. Это, конечно, ухудшит обучение GPT, если обучаться на парах desc-ans целиком. Поэтому я собираюсь в пару к вопросу пациента брать только первое сообщение из столбца ans.
3. В качестве отзывов на лекарства собираюсь взять rudrec_annotated.json, только тексты без NER-разметки.

## Архитектура чат-бота
1. **Классификация интентов**. 

Для классификации интентов собираюсь попробовать 2 подхода - классический через лемматизацию и sklearn-векторайзеры, и нейросетевой подход. Причем есть предположение, что 2-й и 3-й интенты классификатор будет очень сильно путать. Можно будет попробовать двухуровневый классификатор - сначала разделяем болталку и медицинскую тематику, потом если медицинская - разделяем на вопрос к врачу и отзыв на лекарство. Если хорошего разделения медицинских тематик так и не получится, то на третий интент сделаю просто триггер на ключевое слово, например "отзыв на лекарство", что и пропишу в приветственном сообщении бота.

2. **Генерация текста.**

Для болталки я собираюсь использовать просто русскоязычный GPT из коробки. Файл Otvety.txt нужен только для классификации интентов.

Для медицинской Q&A-системы я собираюсь обучить какой-нибудь генератор на датасете medical_qa_ru_data. Для обучения буду пересобирать датасет по аналогии, как было в методичке про диалоговый GPT - брать вопрос из столбца desc и помечать, как пациента, и брать первый ответ из столбца ans и помечать, как врача.

Для подбора отзыва на лекарство буду получать с помощью BERT эмеддинг запроса и находить ближайшие эмбеддинги отзывов из датасета rudrec_annotated. В чат буду выводить n ближайших отзывов, например 3.